##### Copyright 2019 The TensorFlow Authors and Geoffrey Fox 2020


In [ ]:
! which python
! python --version
! pwd

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for theRunTFTCustomVersion Epoch specific language governing permissions and
# limitations under the License.

In [ ]:
from cloudmesh.common.util import banner

import os
import sys
import pwd
import socket
import pathlib
import humanize

from cloudmesh.common.console import Console
from cloudmesh.common.Shell import Shell
from cloudmesh.common.dotdict import dotdict
from cloudmesh.common.Printer import Printer
from cloudmesh.common.StopWatchMllog import progress
from cloudmesh.common.StopWatchMllog import StopWatch
from cloudmesh.common.util import readfile

from cloudmesh.gpu.gpu import Gpu
from pprint import pprint
import sys

from IPython.display import display
import tensorflow_datasets as tfds
import tensorflow as tf
from tqdm.keras import TqdmCallback
from tqdm import tnrange
from tqdm import notebook
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Dense

import os
import subprocess
import gc
from csv import reader
from csv import writer
import sys
import random
import math
import numpy as np
import matplotlib.pyplot as plt
from textwrap import wrap
import pandas as pd
import io as io
import string

import time
import datetime
from datetime import timedelta
from datetime import date
# TODO: better would be to distinguish them and not overwrite the one datetime with the other.
from datetime import datetime
import yaml

from typing import Dict
from typing import Tuple
from typing import Optional
from typing import List
from typing import Union
from typing import Callable

import matplotlib
import matplotlib.patches as patches
# import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.path import Path
import matplotlib.dates as mdates
import scipy as sc
import scipy.linalg as solver
from matplotlib import colors
import enum
import pandas as pd
import abc
import json
import psutil

gregor = True

%load_ext autotime

StopWatch.activate_mllog()
content = readfile("config.yaml")
config = yaml.safe_load(content)

# StopWatch.organization_mllog(**config)
StopWatch.organization_submission(**config)

filename = './earthquake-fig-2.log'
filename = Shell.map_filename(filename).path
print(filename)
Shell.rm(filename)

# StopWatch_Original.progress(0, filename=filename)
progress(progress=0, filename=filename)
StopWatch.start("total", mllog_key="RUN_START")

print(Printer.attribute(config))

In [ ]:

try:
    in_interactive_notebook = config["system"]["notebook"]["interactive"]
except KeyError:
    in_interactive_notebook = False

in_colab = 'google.colab' in sys.modules

in_rivanna = config["system"]["host"] == "rivanna"
#in_rivanna = "hpc.virginia.edu" in socket.getfqdn()

in_dgxstation = config["system"]["host"] == "dgxstation"
in_ubuntu = config["system"]["host"] == "ubuntu"
in_summit = config["system"]["host"] == "summit"

if 'colortheme' in config:
    colortheme = config['colortheme'] == "True"
else:
    colortheme = True


if in_colab:
    # Avoids scroll-in-the-scroll in the entire Notebook
    #test
    from IPython.display import Javascript
    def resize_colab_cell():
      display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 20000})'))
    get_ipython().events.register('pre_run_cell', resize_colab_cell)

    from google.colab import drive
    drive.mount('/content/gdrive')

if in_rivanna or gregor or in_ubuntu or in_dgxstation or in_summit:
    tf.config.set_soft_device_placement(config['set_soft_device_placement'])
    tf.debugging.set_log_device_placement(config['debugging_set_log_device_placement'])


# tf.test.is_gpu_available


In [ ]:
def TIME_start(name, mllog_key="BLOCK_START", **kwargs):
    banner(f"Start timer {name}")
    StopWatch.start(name, mllog_key=mllog_key, **kwargs)

def TIME_stop(name, mllog_key="BLOCK_STOP", **kwargs):
    StopWatch.stop(name, mllog_key=mllog_key, **kwargs)
    t = StopWatch.get(name)
    h = humanize.naturaldelta(timedelta(seconds=t))
    banner(f"Stop timer {name}: {t}s or {h}")


In [ ]:
# who am i

config['user'] = Shell.run('whoami').strip()
try:
    config['user_id'] = Shell.run('id -u').strip()
    config['group_id'] = Shell.run('id -g').strip()
except subprocess.CalledProcessError:
    print("The command <id> is not on your path.")

print(Printer.attribute(config))

In [ ]:
StopWatch.benchmark()

In [ ]:
r = Gpu().system()

try:
    ## Once Cloudmesh GPU PR2 is merged, the above block can be removed and the below be used.
    config['gpuname'] = [x['product_name'] for x in r]
    config['gpuvendor'] = [x.get('vendor', "Unknown Vendor") for x in r]
except:
    pass

print (Printer.attribute(config))

In [ ]:
%matplotlib inline

def SAVEFIG(fig, filename, path=None, formats=('png', 'pdf'), bbox_inches="tight"):

    if "data.images" in config:
        directory = config["data"]["images"]
    else:
        directory = "images"

    if not os.path.exists(directory):
        Shell.mkdir(directory)

    local_path = directory

    fileout = f"{local_path}/{filename}".replace("\n", " ").replace(">", " ").replace("<", " ")
    for my_format in formats:
        fig.savefig(f"{fileout}.{my_format}", format=my_format, bbox_inches=bbox_inches)
    return fig, tuple(f'{fileout}.{fmt}' for fmt in formats)

from matplotlib.transforms import Bbox

#https://stackoverflow.com/questions/4325733/save-a-subplot-in-matplotlib
def full_extent(ax, pad=0.0):
    """Get the full extent of an axes, including axes labels, tick labels, and
    titles."""
    # For text objects, we need to draw the figure first, otherwise the extents
    # are undefined.
    ax.figure.canvas.draw()
    items = ax.get_xticklabels() + ax.get_yticklabels()
#    items += [ax, ax.title, ax.xaxis.label, ax.yaxis.label]
    items += [ax, ax.title]
    bbox = Bbox.union([item.get_window_extent() for item in items])

    return bbox.expanded(1.0 + pad, 1.0 + pad)

In [ ]:
# Set Runname
RunName = 'EARTHQ-newTFTv29'
RunComment = ' TFT Dev on EarthQuakes --  2 weeks 3 months 6 months 1 year d_model 160  dropout 0.1 Location Based Validation BS 64 Simple 2 layers CUDA LSTM MSE corrected MHA Plot 28'

#
# StopWatch.benchmark(sysinfo=True)
# make sure we have free memory in it
# replace the following and if needed read from StopWatch
memory = StopWatch.get_sysinfo()["mem.available"]
print(f'Your runtime has {memory} of available RAM\n')

config['runname'] = RunName

In [ ]:
progress(progress=1, filename=filename)

# Initial System Code

In [ ]:
TIME_start("INIT", mllog_key="INIT_START")
if colortheme:
    startbold = "\033[1m"
    resetfonts = "\033[0m"
    startred = '\033[31m'

    startpurple = '\033[35m'
    startyellowbkg = '\033[43m'
else:
    startbold = ""
    resetfonts = ""
    startred = ''

    startpurple = ''
    startyellowbkg = ''

banner("System information")
r = StopWatch.systeminfo()
print (r)

In [ ]:

banner("nvidia-smi")
gpu_info = Shell.run("nvidia-smi")
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
else:
  print(gpu_info)



# Transformer model for science data based on original for language understanding

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/text/transformer">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/text/transformer.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/transformer.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/text/transformer.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Science Data Parameters and Sizes

-------
Here is structure of science time series module. We will need several arrays that will need to be flattened at times. Note Python defaults to row major i.e. final index describes contiguous positions in memory


At highest level data is labeled by Time and Location

*   Ttot is total number of time steps
*   Tseq is length of each sequence in time steps
*   Num_Seq is number of sequences in time: Num_Seq = Ttot-Tseq + 1
*   Nloc is Number of locations. The locations could be a 1D list or have array structure such as an image.
*   Nsample is number of data samples Nloc * Num_Seq




Input data is at each location
*   Nprop time independent properties describing the location
*   Nforcing is number of time dependent forcing features INPUT at each time value


Output (predicted) data at each location and for each time sequence is
*   Npred predicted time dependent values defined at every time step
*   Recorded at Nforecast time values measured wrt final time value of sequence
*   ForecastDay is an array of length Nforecast defining how many days into future prediction is. Typically ForecastDay[0] = 1 and Nforecast is often 1
*   There is also a class of science problems that are more similar to classic Seq2Seq. Here Nforecast = Tseq and ForecastDay = [-Tseq+1 ... 0]
*   We also support Nwishful predictions of events in future such probability of an earthquake of magnitude 6 in next 3 years. These are defined by araays EventType and Timestart, TimeInterval of length Nwishful. EventType is user defined and Timestart, TimeInterval is measured in time steps
*   Any missing output values should be set to NaN and Loss function must ensure that these points are ignored in derivative calculation and value calculation

We have an input module that supports either LSTM or Transformer (multi-head attention) models

Example Problem AICov

*   Ttot = 114
*   Tseq = 9
*   Num_Seq = 106
*   Nloc = 110


*   Nprop = 35
*   Nforcing = 5 including infections, fatalities, plus 3 temporal position variables (last 3 not in current version)
 
 
*   Npred = 2 (predicted infections and fatalities). Could be 5 if predicted temporal position of output)
*   Nforecast= 15
*   ForecastDay = [1, 2, .......14, 15]
*   Nwishful = 0




















## Science Data Arrays

Typical Arrays


[ time, Location ] as Pandas array with label [name of time-dependent variable] as an array or just name of Pandas array

time labels rows indexed by datetime or the difference datetime - start

Non windowed data is stored with propert name as row index and location as column index
[ static property, Location]

Covid Input is
[Sequence number 0..Num_Seq-1 ] [ Location 0..Nloc-1 ] [position in time sequence Tseq]  [ Input Features]

Covid Output is 
[Sequence number Num_Seq ] [ Location Nloc ]  [ Output Features] 

Output Features are [ ipred = 0 ..Npred-1 ] [ iforecast = 0 ..Nforecast-1 ]

Input Features are static fields followed by if present by dynamic system fields (cos-theta sin-theta linear) chosen followed by cases, deaths. In fact this is user chosen as they set static and dynamic system properties to use

We will have various numpy and pandas arrays where we designate label

[Ttot] is all time values 

[Num_Seq]  is all sequences of window size ***Tseq***

We can select time values or sequences [Ttot-reason] [Num_Seq-reason] for a given "reason"

[Num_Seq][Tseq] is all time values in all sequences

[Nloc] is all locations while [Nloc-reason] is subset of locations for given "reason"

[Model1] is initial embedding of each data point

[Model1+TrPosEnc] is initial embedding of each data point with Transformer style positional encoding

[Nforcing] is time dependent input parameters and [Nprop] static properties while [ExPosEnc] are explicit positional (temporal) encoding.

[Nforcing+ExPosEnc+Nprop] are all possible inputs

[Npred] is predicted values with [Npred+ExPosEnc] as predictions plus encodings with actually used [Predvals] = [Npred+ExPosEnc-Selout] 

[Predtimes] = [Forecast time range] are times forecasted with "time range" separately defined



### Define Basic Control parameters

In [ ]:
def wraptotext(textinput,size=None):
  if size is None:
    size = 120
  textlist = wrap(textinput,size)
  textresult = textlist[0]
  for itext in range(1,len(textlist)):
    textresult += '\n'+textlist[itext]
  return textresult

def timenow():
  now = datetime.now()
  return now.strftime("%m/%d/%Y, %H:%M:%S") + " UTC"

def float32fromstrwithNaN(instr):
  if instr == 'NaN':
    return NaN
  return np.float32(instr)

def printexit(exitmessage):
  print(exitmessage)
  sys.exit()

def strrnd(value):
  return str(round(value,4))

NaN = np.float32("NaN")

ScaleProperties = False
ConvertDynamicPredictedQuantity = False
ConvertDynamicProperties = True
GenerateFutures = False
GenerateSequences = False
PredictionsfromInputs = False
RereadMay2020 = False
UseOLDCovariates = False
Dropearlydata = 0
NIHCovariates = False 
UseFutures = True
Usedaystart = False 
PopulationNorm = False
SymbolicWindows = False
Hydrology = False
Earthquake = False
EarthquakeImagePlots = False
AddSpecialstoSummedplots = False
UseRealDatesonplots = False
Dumpoutkeyplotsaspics = False
OutputNetworkPictures = False
NumpredbasicperTime = 2
NumpredFuturedperTime = 2
NumTimeSeriesCalculated = 0
Dailyunit = 1
TimeIntervalUnitName = 'Day'
InitialDate = datetime(2000,1,1)
NumberofTimeunits = 0
Num_Time =0
FinalDate = datetime(2000,1,1)
GlobalTrainingLoss = 0.0
GlobalValidationLoss = 0.0

# Type of Testing
LocationBasedValidation = False
LocationValidationFraction = 0.0
LocationTrainingfraction = 1.0
RestartLocationBasedValidation = False

global SeparateValandTrainingPlots
SeparateValandTrainingPlots = True
Plotsplitsize = -1 # if > 1 split time in plots

GarbageCollect = True
GarbageCollectionLimit = 0

current_time = timenow()
print(startbold + startred + current_time + ' ' + f'{RunName} {RunComment}' + resetfonts)

Earthquake = True


## Define input structure

Read in data and set it up for Tensorflow with training and validation


Set train_examples, val_examples as science training and validatioon set.

The shuffling of Science Data needs some care. We have ***Tseq*** * size of {[Num_Seq][Nloc]} locations in each sample. In simplease case the last is just a decomposition over location; not over time. Let's Nloc-sel be number of locations per sample. It will be helpful if Nloc-sel is divisable by 2. 

Perhaps Nloc-sel = 2 6 or 10 is reasonable.

Then you shuffle locations every epoch and divide them into groups of size Nloc-sel with 50% overlap so you get locations

0 1 2 3 4 5; 

3 4 5 6 7 8; 

6 7 8 9 10 11 etc.

Every locations appears twice in an epoch (for each time value). You need to randomly add locations at end of sequence so it is divisiuble by Nloc-sel e.g add 4 random positions to the end if Nloc=110 and Nloc-sel = 6. Note last group of 6 has members 112 113 114 0 1 2

After spatial structure set up, randomly shuffle in Num_Seq where there is an argument to do all locations for a partcular time value together.

For validation, it is probably best to select validation location before chopping them into groups of size Nloc-sel

How one groups locations for inference is not clear. One idea is to take trained network and use it to find for each location which other locations have the most attention with it. Use those locations in  prediction

More general input. 
NaN allowed value

* Number time values
* Number locations
* Number driving values
* Number predicted values

For COVID driving same as predicted

* a) Clean up >=0 daily
* b) Normalize
* c) Add Futures
* d) Add time/location encoding


### Setup File Systems

In [ ]:
if in_colab:
    # find the data
    COLABROOTDIR="/content/gdrive/My Drive/Colab Datasets"
elif in_interactive_notebook:
    COLABROOTDIR = config["system"]["notebook"]["colabrootdir"]
else:
    # base_path = f'{config["run"]["workdir"]}'
    base_path = f'{config["run"]["filesystem"]}/mlcommons' \
                f'/{config["experiment"]["TFTTransformerepochs"]}' \
                f'/{config["experiment"]["repeat"]}'

    COLABROOTDIR = f'{base_path}/{config["run"]["datadir"]}'

print (COLABROOTDIR)

if not os.path.exists(COLABROOTDIR):
    Console.error(f"Missing data directory: {COLABROOTDIR}")
    sys.exit(1)

if not os.path.exists(f'{COLABROOTDIR}/EarthquakeDec2020/Outputs'):
    Shell.mkdir(f'{COLABROOTDIR}/EarthquakeDec2020/Outputs')

os.environ["COLABROOTDIR"] = COLABROOTDIR
APPLDIR=f"{COLABROOTDIR}/EarthquakeDec2020"
GRAPHICSDIR=f"{APPLDIR}/Outputs"
CHECKPOINTDIR = f"{APPLDIR}/checkpoints/{RunName}dir/"
Shell.mkdir(CHECKPOINTDIR)
print(f'Checkpoint set up in directory {CHECKPOINTDIR}')

config['checkpointdir'] = CHECKPOINTDIR
config['appldir'] = APPLDIR
config['colabrootdir'] = COLABROOTDIR

print(Printer.attribute(config))
TIME_stop("INIT", mllog_key="INIT_STOP")

### Space Filling Curves

In [ ]:

StopWatch.event("gilbert2d_Graph_Done")
StopWatch.event("faults_ndarray_Graph_Done")



# CELL READ DATA

In [ ]:

# Daily Read in Version
TIME_start("EVAL", mllog_key="EVAL_START")
TIME_start("PLOTS")
TIME_start("CELL_READ_DATA")
TIME_start("SPACE_FILLING_CURVE")
TIME_stop("SPACE_FILLING_CURVE")
StopWatch.event("Observables_summed_over_space_Plotted")
TIME_stop("CELL_READ_DATA")

### Earthquake Eigensystems

In [ ]:
progress(progress=2, filename=filename)

### Multiplicity Data

In [ ]:
TIME_start("MULTIPLICITY_DATA")


In [ ]:

#StopWatch.stop(f"QRegions_Graph_Done_jplot", mllog_key="EVAL_STOP")

StopWatch.event(f"QRegions_Graph_Done_jplot")
StopWatch.event(f"_Graph_Done event_num:")

StopWatch.event(f"_Graph_Done")

StopWatch.event(f"2_Graph_Done")

StopWatch.event(f"3_Graph_Done")


StopWatch.event(f"4_Graph_Done")


StopWatch.event(f"5_Graph_Done")


TIME_stop("MULTIPLICITY_DATA")
# progress

### End of Earthquake. Reset Timing

In [ ]:
progress(progress=10, filename=filename)

### Set Earthquake Execution Mode

### Plot Earthquake Images

In [ ]:
StopWatch.event(f"fault_discovery")

StopWatch.event('Full_Magnitude_Ranges')

StopWatch.event(f'Local_Magnitude_Ranges_')

StopWatch.event("PLOTTED_EARTHQUAKE_IMAGES")
TIME_stop("PLOTS")

### Read and setup NIH Covariates August 2020 and January, April 2021 Data

new collection of time dependent covariates (even if constant).

cases and deaths and location property from previous data


## Process Input Data  in various ways



### Set TFT Mode

### Convert Cumulative to Daily

### Normalize All Static and Dynamic Properties

for Static Properties BasicInputStaticProps[Nloc,NpropperTimeStatic] converts to NormedInputStaticProps[Nloc,NpropperTimeStatic]

### Set up Futures

-- currently at unit time level 

### Set up mappings of locations

In next cell, we map locations for BEFORE location etc added

In cell after that we do same for sequences

###Property and Prediction  Data Structures

Two important Lists Properties and Predictions that are related

 * Data stored in series is for properties, the calculated value occuring at or ending that day
 * For predictions, the data is the calculated value from that date or later. 

 * We store data labelled by time so that
  * for inputs we use time 0 upto last value - 1 i.e. position [length of array - 1]
  * for outputs (predictions) with sequence Tseq, we use array locations [Tseq] to [length of array -1]
  * This implies Num_Seq = Num_Time - Tseq


**Properties**

Everything appears in Property list -- both input and output (predicted)
DynamicPropertyTimeSeries holds input property time series where value is value at that time using data before this time for aggregations
  * NpropperTimeStatic is the number of static properties -- typically read in or calculated from input information
  * NpropperTimeDynamicInput is total number of input time series
  * NpropperTimeDynamicCalculated is total number of calculated dynamic quantities  used in Time series analysis as input properties and/or output predictions
  * NpropperTimeDynamic = NpropperTimeDynamicInput + NpropperTimeDynamicCalculated ONLY includes input properties
  * NpropperTime = NpropperTimeStatic + NpropperTimeDynamic will not include futures and NOT include calculated predictions
  * InputPropertyNames is a list of size NpropperTime holding names
  * NpropperTimeMAX = NpropperTime + NumTimeSeriesCalculated has calculated predictions following input properties ignoring futures 
  * QuantityStatistics has 7 statistics used in normalizing for NpropperTimeMAX properties
  * Normalization takes NpropperTimeStatic static features in BasicInputStaticProps and stores in NormedInputStaticProps
  * Normalization takes NpropperTimeDynamicInput dynamic features in BasicInputTimeSeries and stores in NormedInputTimeSeries
  * Normalization takes NpropperTimeDynamicCalculated dynamic features in DynamicPropertyTimeSeries and stores in NormedDynamicPropertyTimeSeries

**Predictions**

 * NumpredbasicperTime can be 1 upto NpropperTimeDynamic and are part of dynamic input series. It includes input values that are to be predicted (these MUST be at start) plus NumTimeSeriesCalculated calculated series
 * NumpredFuturedperTime is <= NumpredbasicperTime and is the number of input dynamic series that are futured
 * NumTimeSeriesCalculated is number of calculated (not as futures) time series stored in CalculatedTimeSeries and names in NamespredCalculated
 * Typically NumpredbasicperTime = NumTimeSeriesCalculated + NumpredFuturedperTime (**Currently this is assumed**)
 * Normalization takes NumTimeSeriesCalculated calculated series in CalculatedTimeSeries and stores in NormedCalculatedTimeSeries
 * Predictions per Time are  NpredperTime = NumpredbasicperTime + NumpredFuturedperTime*LengthFutures
 * Predictions per sequence Npredperseq = NpredperTime



### Set Requested Properties Predictions Encodings

### Choose Input and Predicted Quantities

### Calculate Futures

Start Predictions

### Set up Predictions 

first for time arrays; we will extend to sequences next. Sequences include the predictions for final time in sequence.

This is prediction for sequence ending one day before the labelling time index. So sequence must end one unit before last time value

Note this is  "pure forecast" which are of quantities used in driving data allowing us to iitialize prediction to input

NaN represents non existent data

### Clean-up Input quantities#################

###Setup Sequences and UseTFTModel

###Generate Sequences from Time labelled data 
given Tseq set above

### Define Possible Temporal and Spatial Positional Encodings

In [ ]:
progress(progress=20, filename=filename)

### Add in Temporal and Spatial Encoding

###Set up NNSE and Plots including Futures

## Location Based Validation

## LSTM Control Parameters EDIT

## Important Parameters defining Transformer project EDIT

## General Control Parameters

In [ ]:

TIME_stop("EVAL")

## Useful Time series utilities

### DLprediction

Prediction and Visualization LSTM+Transformer

In [ ]:
StopWatch.start('DLprediction Iteration')
StopWatch.stop('DLprediction Iteration')
StopWatch.start('DLprediction Model Fit')
StopWatch.stop('DLprediction Model Fit')
StopWatch.start('DLprediction Performance Loss')
StopWatch.stop('DLprediction Performance Loss')
StopWatch.event(f"RunTFTCustomVersion MSE_")


### Spatial Earthquake Plots

###Organize Location v Time Plots

### Summed & By Location Plots

In [ ]:
TIME_start(f"DLResults_Graphs timer_num")
StopWatch.event(f"RunTFTCustomVersion DLResults Graph Title event_num:")

TIME_stop(f"DLResults_Graphs timer_num")
StopWatch.event(f"prediction_plot_ event_num:")
StopWatch.event(f"earthquake_pred_ event_num:")

StopWatch.event(f"predvsreal_ event_num:")


In [ ]:
progress(progress=40, filename=filename)

### Calculate




In [ ]:

StopWatch.event(f"NNSE event_num:")


### Custom Loss Functions

### Utility: Shuffle, Finalize


In [ ]:
StopWatch.event(f"_Graph_Done event_num:")


### DLPrediction2E printloss ?DEL

### DLPrediction2F Sensitivity

# TFT Model

##Set up TFT

###Data and Input Types

###Convert FFFFWNPF to TFT

In [ ]:

StopWatch.event(f"Predicted_Data_")
StopWatch.event(f"TFTdfTotalSpec event_num:")



###TFT Setup

###Base Formatter

###TFT FFFFWNPF Formatter

###Set TFT Parameter Dictionary

In [ ]:
progress(progress=60, filename=filename)

###TFTTools

### Setup  Classic TFT

###Read TFT Data

In [ ]:
StopWatch.start("data head setup train")
StopWatch.stop("data head setup train")
StopWatch.start("data head setup valid")
StopWatch.stop("data head setup valid")
StopWatch.start("data head setup test")
StopWatch.stop("data head setup test")
StopWatch.start("data head setup")
StopWatch.stop("data head setup")


##Predict TFT 

###Visualize TFT

Called from finalizeDL

In [ ]:
TIME_start(f"RunTFTCustomVersion bestfit finalize VisualizeTFT TFTSaveandInterpret setFFFFmapping timer_num:")
TIME_stop(f"RunTFTCustomVersion bestfit finalize VisualizeTFT TFTSaveandInterpret setFFFFmapping timer_num:")
TIME_start(f"RunTFTCustomVersion bestfit finalize VisualizeTFT DLprediction timer_num:")
TIME_stop(f"RunTFTCustomVersion bestfit finalize VisualizeTFT DLprediction timer_num:")


##TFT Routines

### GLUplusskip: Gated Linear unit plus add and norm with Skip

###Linear Layer (Dense)

###Apply MLP

###GRN Gated Residual Network

###Process Static Variables

###Process Dynamic Variables

###TFT LSTM

###TFT Multihead Temporal Attention

###TFTFullNetwork

In [ ]:
progress(progress=70, filename=filename)

##TFT Run & Output

### General Utilities

### Tensorflow  Monitor

In [ ]:

StopWatch.event(f"RunTFTCustomVersion Epoch: event_num:")

StopWatch.event(f"RunTFTCustomVersion Last Loss Best Loss event_num:")
StopWatch.event(f"RunTFTCustomVersion Best Fit event_num:")
StopWatch.event(f"RunTFTCustomVersion Best Fit event_num:")


###TFT Output

In [ ]:

StopWatch.event(f"RunTFTCustomVersion bestfit finalize make graph event_num:")
TIME_start(f"RunTFTCustomVersion bestfit finalize TFTTestpredict timer_num:")
TIME_stop(f"RunTFTCustomVersion bestfit finalize TFTTestpredict timer_num:")
TIME_start(f"RunTFTCustomVersion bestfit finalize VisualizeTFT timer_num:")
StopWatch.event(f"RunTFTCustomVersion bestfit finalize VisualizeTFT event_num:")
TIME_stop(f"RunTFTCustomVersion bestfit finalize VisualizeTFT timer_num:")


###TFTcustommodel

Control Full TFT Network

### TFT Overall Batch Training

* TIME not set explicitly
* Weights allowed or not
* Assumes TFTFullNetwork is full Network

In [ ]:

TIME_start(f"RunTFTCustomVersion init timer_num:", mllog_key="BLOCK_START")

TIME_stop(f"RunTFTCustomVersion init timer_num:", mllog_key="BLOCK_STOP")

TIME_start(f"RunTFTCustomVersion restore timer_num:", mllog_key="BLOCK_START")
TIME_stop(f"RunTFTCustomVersion restore timer_num:")

TIME_start(f"RunTFTCustomVersion analysis timer_num:", mllog_key="BLOCK_START")
TIME_stop(f"RunTFTCustomVersion analysis timer_num:", mllog_key="BLOCK_STOP")
TIME_start(f"RunTFTCustomVersion train timer_num:", mllog_key="EVAL_START")
TIME_start(f"RunTFTCustomVersion train Epoch: timer_num:", mllog_key="EPOCH_START")
StopWatch.event(f"RunTFTCustomVersion train Epoch:-start event_num:")
StopWatch.start('DLTFTCustom TrainTest Split')
StopWatch.stop('DLTFTCustom TrainTest Split')
StopWatch.start('DLTFTCustom Reshape Data')
StopWatch.stop('DLTFTCustom Reshape Data')
StopWatch.start('DLTFTCustom Train Data')
StopWatch.stop('DLTFTCustom Train Data')
StopWatch.event(f"RunTFTCustomVersion train Epoch:-stop event_num:")
TIME_stop(f"RunTFTCustomVersion train Epoch: timer_num:", mllog_key="EPOCH_STOP")

TIME_start(f"RunTFTCustomVersion validation Epoch: timer_num:", mllog_key="EVAL_START")
TIME_start(f"RunTFTCustomVersion validation bestfit Epoch: timer_num:")
StopWatch.start('TFTSymbolicWindows')
StopWatch.start('label1')
StopWatch.stop('label1')
StopWatch.start('label3')
StopWatch.stop('label3')
StopWatch.start('label2')
StopWatch.stop('label2')
StopWatch.stop('TFTSymbolicWindows')

TIME_stop(f"RunTFTCustomVersion validation Epoch: timer_num:", mllog_key="EVAL_STOP")

# Decide on best fit
TIME_start(f"RunTFTCustomVersion bestfit Epoch: timer_num:")

TIME_stop(f"RunTFTCustomVersion bestfit Epoch: timer_num:")
TIME_stop(f"RunTFTCustomVersion validation bestfit Epoch: timer_num:")
TIME_stop(f"RunTFTCustomVersion bestfit Epoch: timer_num:")
TIME_stop(f"RunTFTCustomVersion validation bestfit Epoch: timer_num:")
TIME_stop(f"RunTFTCustomVersion train timer_num:", mllog_key="EVAL_STOP")
TIME_start(f"RunTFTCustomVersion bestfit timer_num:", mllog_key="BLOCK_START")

TIME_start(f"RunTFTCustomVersion bestfit TFTTrainingMonitor timer_num:", mllog_key="BLOCK_START")
TIME_stop(f"RunTFTCustomVersion bestfit TFTTrainingMonitor timer_num:", mllog_key="BLOCK_STOP")

TIME_start(f"RunTFTCustomVersion bestfit Checkpointfinalstate timer_num:", mllog_key="BLOCK_START")
TIME_stop(f"RunTFTCustomVersion bestfit Checkpointfinalstate timer_num:", mllog_key="BLOCK_STOP")
StopWatch.event(f"RunTFTCustomVersion Train Loss Val Loss event_num:")
TIME_start(f"RunTFTCustomVersion bestfit summary timer_num:")
TIME_stop(f"RunTFTCustomVersion bestfit summary timer_num:")

TIME_start(f"RunTFTCustomVersion bestfit network summary timer_num:")
TIME_stop(f"RunTFTCustomVersion bestfit network summary timer_num:")


TIME_start(f"RunTFTCustomVersion bestfit TFTLSTMEncoder summary timer_num:")
TIME_stop(f"RunTFTCustomVersion bestfit TFTLSTMEncoder summary timer_num:")

TIME_start(f"RunTFTCustomVersion bestfit TFTLSTMDecoder summary timer_num:")
TIME_stop(f"RunTFTCustomVersion bestfit TFTLSTMDecoder summary timer_num:")

TIME_start(f"RunTFTCustomVersion bestfit Network attn layer summary timer_num:")
TIME_stop(f"RunTFTCustomVersion bestfit Network attn layer summary timer_num:")

TIME_start(f"RunTFTCustomVersion bestfit Network attn layer attention summary timer_num:")
TIME_stop(f"RunTFTCustomVersion bestfit Network attn layer attention summary timer_num:")
TIME_start(f"RunTFTCustomVersion bestfit Model build graph timer_num:")
TIME_stop(f"RunTFTCustomVersion bestfit Model build graph timer_num:")

TIME_start(f"RunTFTCustomVersion bestfit Network build graph timer_num:")
TIME_stop(f"RunTFTCustomVersion bestfit Network build graph timer_num:")

TIME_start(f"RunTFTCustomVersion bestfit finalize timer_num:")

TIME_stop(f"RunTFTCustomVersion bestfit finalize timer_num:")

TIME_stop(f"RunTFTCustomVersion bestfit timer_num:", mllog_key="BLOCK_STOP")



###Run TFT

In [ ]:
# Run TFT Only
TIME_start("RunTFTCustomVersion tft only")

TIME_stop("RunTFTCustomVersion tft only")


In [ ]:
progress(progress=80, filename=filename)

In [ ]:
TIME_start("RunTFTCustomVersion print")
TIME_stop("RunTFTCustomVersion print")

TIME_start("RunTFTCustomVersion A")
TIME_stop("RunTFTCustomVersion A")

In [ ]:
StopWatch.stop("total", mllog_key="RUN_STOP")
StopWatch.benchmark()

In [ ]:
import re

#StopWatch.event(config)
#StopWatch.event(f"gpu={}")
#StopWatch.benchmark(sysinfo=False, tag="This_is_final")
StopWatch.benchmark(sysinfo=False, attributes="short")

if in_rivanna:
    print(str(sys.executable))
    print("Run in Rivanna")
    test_localscratch = re.search('localscratch', str(sys.executable))
    test_scratch = re.search('scratch', str(sys.executable))
    test_project = re.search('project', str(sys.executable))
    if test_localscratch != None:
        print('Working Directory = ' + str(test_localscratch.group(0)))
    if test_scratch != None:
        print('Working Directory = ' + str(test_scratch.group(0)))
    if test_project != None:
        print('Working Directory = ' + str(test_project.group(0)))
    print('Number of Epochs = ' + str(config['experiment']['TFTTransformerepochs']))
    print('Card = ' + str(config['experiment']['card_name']))
    print("Partition is " + str(os.getenv('SLURM_JOB_PARTITION')))
    print("Job ID is " + str(os.getenv("SLURM_JOB_ID")))

In [ ]:
StopWatch.status_submission(success=True)
progress(progress=100, filename=filename)

In [ ]:
sys.exit(0)

#End modified TFT